In [0]:
import requests
import pandas as pd
from datetime import datetime

url = f"https://api.coingecko.com/api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=90"
response = requests.get(url)
data = response.json()

# Convert 'prices', 'market_caps', 'total_volumes' into DataFrames
df_prices = pd.DataFrame(data['prices'], columns=['timestamp', 'price'])
df_market_cap = pd.DataFrame(data['market_caps'], columns=['timestamp', 'market_cap'])
df_volume = pd.DataFrame(data['total_volumes'], columns=['timestamp', 'volume'])

# Merge all into a single DataFrame on timestamp
df = df_prices.merge(df_market_cap, on='timestamp').merge(df_volume, on='timestamp')

# Convert timestamp to datetime
df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

# Set timestamp as index (good for time series)
df.set_index('timestamp', inplace=True)

# Display result
print(df.head())

                                price    market_cap        volume
timestamp                                                        
2025-02-05 09:03:58.345  97656.245281  1.935505e+12  6.096673e+10
2025-02-05 10:03:50.353  97450.196133  1.929595e+12  6.534012e+10
2025-02-05 11:03:52.815  97759.883175  1.939221e+12  6.366141e+10
2025-02-05 12:01:03.860  98193.323038  1.947088e+12  6.252212e+10
2025-02-05 13:02:35.879  98045.781830  1.943194e+12  6.356655e+10


(2160, 3)